Nikhil Adithyan
Aug 1, 2023 /7 min read

I’ve always been a huge fan of TradingView’s charting tool, especially for its beautifully crafted user interface and design. And there’s never been a day I haven’t thought about recreating the graph design in Python.
But, it’s always tough for Python developers to create stunning and professional-looking visualizations (like TradingView) using libraries like Matplotlib, Seaborn, Altair, etc. Their style themes are so outdated and crappy. Though all these modules provide features for customizing the theme of the charts, it takes a toll on the developer as there is a lot of work involved.


Fortunately, I recently came across an awesome library called lightweight-charts-python providing features to easily re-create the TradingView style with minimal code. In this article, we will dive deep into this library, explore its features, and code some cool TradingView charts in Python.

Importing Packages
The first and foremost step of setting up the coding environment is to import the required packages. In this article, we are going to use five different packages which are pandas for data manipulation, and requests for making API calls, numpy for numerical calculations, lightweight_chart for replicating the TradingView look, time for time-related functions, and finally asyncio and nest_asyncio for asynchronous programming. The following code will import all the mentioned packages into our Python environment:

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio
nest_asyncio.apply()



Obtaining Data using Yfinance

In [2]:
import yfinance as yf
df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)

df.head()

C:\Users\jwang\AppData\Local\Temp\ipykernel_14264\3415087877.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,21.760860,21.760860,21.133237,21.250616,65536000
1,2020-01-03,21.169172,21.559641,20.898478,20.910456,72590000
2,2020-01-06,21.571617,21.578803,20.589454,20.666111,64047600
3,2020-01-07,21.545267,21.746490,21.358417,21.593177,53849600
4,2020-01-08,22.041140,22.345369,21.447051,21.550058,79582400


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]

In [4]:
ema_results_25 = indicators.get_ema(quotes, 25)
df['EMA 25'] = [r.ema for r in ema_results_25]
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]
df = df.dropna().reset_index(drop=True)
df.head()

,Date,Close,High,Low,Open,Volume,EMA 25,EMA 12,EMA 20,upper_band,middle_band,lower_band,rsi,rsima6,rsima12,dynamic20
0,2020-02-07,25.636806,26.065604,25.421212,25.627226,77026000,23.398761,24.313799,23.778778,25.831383,23.762914,21.694446,66.386630,62.719491,62.560073,23.346677
1,2020-02-10,26.561476,26.568664,25.368510,25.387675,61624400,23.642047,24.659596,24.043797,26.274287,23.970366,21.666446,70.134168,65.963315,61.989442,23.506583
2,2020-02-11,26.580643,27.277739,26.369838,26.997462,96885200,23.868093,24.955141,24.285401,26.676574,24.139610,21.602645,70.208312,68.073415,62.232093,23.663267
3,2020-02-12,27.359179,27.392716,26.853725,27.042971,60804000,24.136638,25.324993,24.578142,27.192866,24.361314,21.529763,73.126664,69.330238,64.060556,23.835624
4,2020-02-13,27.253777,27.572381,26.681249,26.779466,77641200,24.376418,25.621729,24.832964,27.593731,24.577030,21.560329,72.096955,70.211432,65.177225,24.002275


In the code, the reason for changing the column names is that lightweight_charts demands a specific naming structure to plot the data. Now that we have adequate data to work with, let’s make some cool visualizations.

In [5]:
if __name__ == '__main__':

    chart = Chart( maximize=True)
    chart.set(df)
    chart.show(block = True)

It just takes as little as three lines of code to create a graph in the look of TradingView’s charting platform. And the code is very straightforward in nature. We are first creating an instance of the class Chart and assigned it to the chart variable. Then using the set function, we are setting the data of the chart. Finally, we are displaying the created chart with the help of the show function. 

This output is absolutely stunning for a program of three lines of code. But people who are disappointed at the output after viewing the thumbnail of this article, don't worry! Because now, we are going to up the game by customizing the whole theme of the plot, adding more details, and simply, making it even more beautiful. Here’s the code to do that:

In [6]:
if __name__ == '__main__':

    chart = Chart(title="TQQQ Stock Price", height = 600, width = 1000, maximize=True)

    chart.grid(vert_enabled = True, horz_enabled = True)

    chart.layout(background_color='#131722', font_family='Trebuchet MS', font_size = 16)

    chart.candle_style(up_color='#2962ff', down_color='#e91e63',
                    border_up_color='#2962ffcb', border_down_color='#e91e63cb',
                    wick_up_color='#2962ffcb', wick_down_color='#e91e63cb')

    chart.volume_config(up_color='#2962ffcb', down_color='#e91e63cb')

    chart.legend(visible = True, font_family = 'Trebuchet MS', ohlc = True, percent = True)

    #####################################################################################

    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    chart.show(block = True)

The code might not be as short as the previous one for the basic plot, but it’s actually very simple. And for easy explanation, I’ve divided the code into two separate parts. The first part is about theme customization. It includes changing the background color, the colors of the candles and volume bars, and so on. Basically, the things related to the style of the plot are dealt with in the first part. The second part is about adding details to the plot. 

This is absolutely fantastic! We fully customized the whole look and feel of the plot and added more details like SMA lines and legends for a more insightful graph.


Now let’s move our focus from historical graphs to another cool feature provided by the lightweight_charts library which is the real-time charting feature. Real-time charts are extremely useful for day traders to keep track of the latest price movements and TradingView is most preferred for such charts. Just like how we replicated the historical charts of TradingView, let’s do the same thing for real-time charts too. This is the code to create a real-time TradingView chart:

In this code, we are not actually using the real-time data of stock prices but rather simulating it using the previously extracted historical data. We are first splitting the historical data into two separate dataframes. The first one is used as the initial data for the plot and the second one is used as the real-time data which is done by updating the data points of the plot with the help of a for-loop. 

Pretty cool, right?! But like how there was a lot of scope for improvements in the basic historical graph, this real-time chart can also be improved and modified in a lot of places. We can first change the theme of the plot and similar to how we added SMA lines to the historical chart for better insights, we can add more details for an informative visualization. Here’s the code for the modified or advanced version of the initial real-time chart:

In [7]:
from lightweight_charts import Chart
import pandas as pd

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    rt_chart = Chart( maximize=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    rt_chart.set(df)

    # Create line series for EMAs
    ema12_line = rt_chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = rt_chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_ema12, p_ema25 = df.iloc[i-1]['EMA 12'], df.iloc[i-1]['EMA 25']
        c_ema12, c_ema25 = df.iloc[i]['EMA 12'], df.iloc[i]['EMA 25']
        c_rsi = df.iloc[i]['rsi']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if p_ema12 < p_ema25 and c_ema12 > c_ema25 and c_rsi < 60:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif p_ema12 > p_ema25 and c_ema12 < c_ema25 and c_rsi > 40:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        rt_chart.marker_list(markers)
    
rt_chart.show(block = True)


In [ ]:
if __name__ == '__main__':

    chart = Chart(title="Mcginley Dynamic 20", maximize=True)
    chart.legend(visible=True)
    chart.set(df)

    mcginley_line = chart.create_line('dynamic20', color='#26c6da', width=1, price_label=True)
    mcginley_line.set(df[['Date', 'dynamic20']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_mcginley, p_close = df.iloc[i-1]['dynamic20'], df.iloc[i-1]['Close']
        c_mcginley, c_close = df.iloc[i]['dynamic20'], df.iloc[i]['Close']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if p_close < p_mcginley and c_close > c_mcginley :
            
        # Calculate the time for the next day
            next_day = current_time + timedelta(days=1)
            
            markers.append({
                'time': next_day,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif p_close > p_mcginley and c_close < c_mcginley :
             
             next_day = current_time + timedelta(days=1)
             
             markers.append({
                'time': next_day,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    chart.show(block = True)

In [9]:
if __name__ == '__main__':
    
    chart = Chart(title="RSI crossove RSIMA12 + Mcginley dynamic 20", inner_width=1, inner_height=0.8, maximize=True)
    chart.legend(visible=True)
    chart.set(df)
    
    mcginley_line = chart.create_line('dynamic20', color="#e4b713", width=1, price_label=True, price_line=False)
    mcginley_line.set(df[['Date', 'dynamic20']])

    chart2 = chart.create_subchart(width=1, height=0.2, sync=True)
    rsi_line = chart2.create_line('rsi', color='green', width=1, price_line=False)
    df2 = df[['Date', 'rsi']]
    rsi_line.set(df2)
    
    rsima_line = chart2.create_line('rsima12', color='red', width=1, price_line=False)
    df3 = df[['Date', 'rsima12']]
    rsima_line.set(df3)

    # Initialize a list to hold the dynamic markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_mcginley, p_close = df.iloc[i-1]['dynamic20'], df.iloc[i-1]['Close']
        c_mcginley, c_close = df.iloc[i]['dynamic20'], df.iloc[i]['Close']
        p_rsi, p_rsima12 = df.iloc[i-1]['rsi'], df.iloc[i-1]['rsima12']
        c_rsi, c_rsima12 = df.iloc[i]['rsi'], df.iloc[i]['rsima12']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if p_close < p_mcginley and c_close > c_mcginley and p_rsi < p_rsima12 and c_rsi > c_rsima12:
            next_day = current_time + timedelta(days=1)
            markers.append({
                'time': next_day,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif p_close > p_mcginley and c_close < c_mcginley and p_rsi > p_rsima12 and c_rsi < c_rsima12:
            next_day = current_time + timedelta(days=1)
            markers.append({
                'time': next_day,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers) 
        
  
    chart.show(block = True)